# Theory reader draft

In [65]:
import re

In [66]:
thm_inits = ['lemma', 'theorem', 'corollary', 'proposition', 'schematic_goal', 'function', 'termination']
prf_inits = ['lemma', 'theorem', 'corollary', 'proposition', 'schematic_goal', 'function', 'termination',
            'by', 'apply', 'using', 'unfolding', 'then have', 'then show', 'have', 'hence', 'show', 'thus', 'case']

thm_init_rgx = re.compile(r'\b(' + '|'.join(map(re.escape, thm_inits)) + r')\b')
prf_init_rgx = re.compile(r'\b(' + '|'.join(map(re.escape, prf_inits)) + r')\b')
prf_end_rgx = re.compile(r'\b(' + '|'.join(map(re.escape, ['by', 'qed', 'done'])) + r')\b')

test_path = 'Test.thy'

def read_file(test_path):
    with open(test_path, 'r') as file:
        lines = file.readlines()
    return "".join(lines)

def proof_starting(s):
    return prf_init_rgx.search(s)

def proof_ending(line):
    return prf_end_rgx.search(line)

In [67]:
def split_on(pattern, s):
    matches = list(re.finditer(pattern, s)) # list of match obj
    chunks = []
    init_pos = 0
    for match in matches:
        match_pos = match.start()
        chunk = s[init_pos:match_pos]
        chunks.append(chunk)
        init_pos = match_pos
    if init_pos < len(s):
        chunks.append(s[init_pos:])
    return chunks

# uses a stack buffer to find the indices of all parent comments
# returns the list of comments and the resulting string without comments
def remove_comments(s):
    i = 0
    stack = []
    parent_idxs = []
    while i < len(s):
        if s[i:i+2] == "(*":
            stack.append(i)
            i += 2
        elif s[i:i+2] == "*)":
            if stack: 
                start = stack[-1]
                stack.pop()
                if not stack:
                    parent_idxs.append((start, i+2))
                i += 2
            else:
                raise Exception("Unbalanced comments")
                break
        else:
            i += 1
    i = 0
    chunks = []
    comments = []
    for idxs in parent_idxs:
        if i < idxs[0]:
            chunks.append(s[i:idxs[0]-1])
        comments.append(s[idxs[0]:idxs[1]])
        i = idxs[1] + 1
    if i < len(s):
        chunks.append(s[i:])
    return " ".join(chunks), comments

In [ ]:
# unfinished
def insert_retrieval_cmnd(file_path):
    thy_str, comments = remove_comments(read_file(file_path))
    prf_matches = list(re.finditer(prf_init_rgx, s))
    init_pos = 0
    for match in prf_matches:
        match_pos = match.start()
        chunk = s[init_pos:match_pos]
        if match.group() in thm_inits:
            chunks.append()
        else:
            chunks.append(chunk)
            init_pos = match_pos
        if init_pos < len(s):
            chunks.append(s[init_pos:])
    return chunks

In [68]:
test_file = read_file(test_path)
# type(test_file)

In [69]:
test_str, comments = remove_comments(test_file)
what = split_on(prf_init_rgx, test_str)
what[1:90]

['lemma eq_map_tree_Leaf[simp]: "map_tree f t = Leaf \\<longleftrightarrow> t = Leaf"\n',
 'by (rule tree.map_disc_iff)\n\n',
 'lemma eq_Leaf_map_tree[simp]: "Leaf = map_tree f t \\<longleftrightarrow> t = Leaf"\n',
 'by (cases t) auto\n\n\nsubsection \\<open>\\<^const>\\<open>size\\<close>\\<close>\n\n',
 'lemma size1_size: "size1 t = size t + 1"\n',
 'by (induction t) simp_all\n\n',
 'lemma size1_ge0[simp]: "0 < size1 t"\n',
 'by (simp add: size1_size)\n\n',
 'lemma eq_size_0[simp]: "size t = 0 \\<longleftrightarrow> t = Leaf"\n',
 'by(cases t) auto\n\n',
 'lemma eq_0_size[simp]: "0 = size t \\<longleftrightarrow> t = Leaf"\n',
 'by(cases t) auto\n\n',
 'lemma neq_Leaf_iff: "(t \\<noteq> \\<langle>\\<rangle>) = (\\<exists>l a r. t = \\<langle>l, a, r\\<rangle>)"\n',
 'by (cases t) auto\n\n',
 'lemma size_map_tree[simp]: "size (map_tree f t) = size t"\n',
 'by (induction t) auto\n\n',
 'lemma size1_map_tree[simp]: "size1 (map_tree f t) = size1 t"\n',
 'by (simp add: size1_size)\n\n\ns

In [70]:
comments

['(* this is a comment (* this is a comment inside a comment with the string theory *) *)',
 '(* this is a comment with a new line,\n (* and a three nested comment (* see! *) *) therefore it should be covered up to below\n *)',
 '(* comment with the string theory *)',
 '(* another comment with the string theory *)',
 '(* another comment with theory *)']

In [71]:
what[0]

'\\<comment> \\<open> another comment with string theory \\<close>\nsection \\<open>A section with string theory\\<close>\n\ntheory    \\<comment> \\<open>and this comment with theory \\<close>\n Test\nimports Main\nbegin\n\ndatatype \'a tree =\n  Leaf ("\\<langle>\\<rangle>") |\n  Node "\'a tree" ("value": \'a) "\'a tree" ("(1\\<langle>_,/ _,/ _\\<rangle>)")\ndatatype_compat tree\n\nprimrec left :: "\'a tree \\<Rightarrow> \'a tree" where\n"left (Node l v r) = l" |\n"left Leaf = Leaf"\n\nprimrec right :: "\'a tree \\<Rightarrow> \'a tree" where\n"right (Node l v r) = r" |\n"right Leaf = Leaf"\n\ntext\\<open>Counting the number of leaves rather than nodes:\\<close>\n\nfun size1 :: "\'a tree \\<Rightarrow> nat" where\n"size1 \\<langle>\\<rangle> = 1" |\n"size1 \\<langle>l, x, r\\<rangle> = size1 l + size1 r"\n\nfun subtrees :: "\'a tree \\<Rightarrow> \'a tree set" where\n"subtrees \\<langle>\\<rangle> = {\\<langle>\\<rangle>}" |\n"subtrees (\\<langle>l, a, r\\<rangle>) = {\\<langle>l, 

In [74]:
# regex experiment
def get_theory_name(s):
    thy_init_rgx = re.compile(r'theory.*?begin', re.S)
    thy_init_match = thy_init_rgx.search(s)
    if thy_init_match:
        thy_init = thy_init_match.group()
        thy_name_rgx = re.compile(r'theory\s*(?:\(\*.*?\*\)\s*|―‹.*?›\s*)*(\S+)', re.S)
        thy_name_match = thy_name_rgx.search(thy_init)
        if thy_name_match:
            thyy_name = thy_name_match.group(1)
            return thyy_name
    return None

In [75]:
test_file = read_file(test_path)
test_str, comments = remove_comments(test_file)
get_theory_name(test_str)

'\\<close>'